In [93]:
%load_ext autoreload
%reload_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf
from pytorch_lightning import seed_everything
import icevision
from icevision.core import *
import hydra

In [2]:
initialize(config_path="configs/")
cfg = compose(config_name='config', overrides=[])
print(cfg)

{'seed': 123, 'name': None, 'ignore_warnings': False, 'trainer': {'_target_': 'pytorch_lightning.Trainer', 'gpus': 4, 'strategy': 'ddp', 'min_epochs': 100, 'max_epochs': 300}, 'module': {'model_type': 'models.ross.efficientdet', 'backbone': 'tf_d4', 'img_size': 1024, 'model': {'_target_': 'src.modules.EfficientDetModule.EfficientDetModule', 'model': {'_target_': 'icevision.${module.model_type}.model', 'backbone': {'_target_': 'icevision.${module.model_type}.backbones.${module.backbone}', 'pretrained': 1}, 'num_classes': 2, 'img_size': '${module.img_size}'}, 'learning_rate': 0.0001}}, 'datamodule': {'_target_': 'src.datamodules.DentalCaries.DentalCariesDataModule', 'data_root': '/home.stud/kuntluka/dataset/carries_dataset', 'image_size': '${module.img_size}', 'model_type': '${module.model_type}', 'batch_size': 1, 'num_workers': 8}, 'logger': {'wandb': {'_target_': 'pytorch_lightning.loggers.wandb.WandbLogger', 'project': 'ice_framework', 'name': '${name}'}}, 'callbacks': {'rich_progress

In [3]:
seed_everything(42)

dm = hydra.utils.instantiate(cfg.datamodule)
# dm
dm.train_tfms = dm.valid_tfms
dm.setup()

Global seed set to 42


  0%|          | 0/3079 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/1626 [00:00<?, ?it/s]

In [4]:
import torch
from icevision.all import *
import hydra

In [5]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [6]:
checkpoint = torch.load('epoch_139.ckpt')
model = hydra.utils.instantiate(cfg.module.model)
model.load_state_dict(checkpoint['state_dict'])
model = model.model

In [32]:
model

DetBenchTrain(
  (model): EfficientDet(
    (backbone): EfficientNetFeatures(
      (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act1): SiLU(inplace=True)
      (blocks): Sequential(
        (0): Sequential(
          (0): DepthwiseSeparableConv(
            (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act1): SiLU(inplace=True)
            (se): SqueezeExcite(
              (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (act1): SiLU(inplace=True)
              (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
              (gate): Sigmoid()
            )
            (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
       

In [7]:
model_type = models.ross.efficientdet
val_dl = dm.val_dataloader()
inf_loader = model_type.infer_dl(dm.valid_ds, batch_size=4, shuffle=False)
inf_loader_train = model_type.infer_dl(dm.train_ds, batch_size = 4, shuffle = False)

# preds = model_type.predict_from_dl(model, inf_loader, keep_images = True, detection_threshold = 0.1)

In [28]:
# preds[0].detection.scores
device = 'cuda'
model = model.to(device)
model.eval()
preds_all = model_type.predict_from_dl(model, inf_loader, keep_images = False, detection_threshold = 0.0)
preds_all_train = model_type.predict_from_dl(model, inf_loader_train, keep_images = False, detection_threshold = 0.0)
# inf_loader.to(device)

  0%|          | 0/82 [00:00<?, ?it/s]

/home.stud/kuntluka/.conda/envs/ice/lib/python3.8/site-packages/effdet/bench.py:45: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  indices_all = cls_topk_indices_all // num_classes


  0%|          | 0/325 [00:00<?, ?it/s]

In [9]:
print(preds[7].ground_truth)
print(preds[7].detection)

BaseRecord

common: 
	- Record ID: 1363.png
	- Filepath: /home.stud/kuntluka/dataset/carries_dataset/images/1363.png
	- Img: None
	- Image size ImgSize(width=1024, height=1024)
detection: 
	- Class Map: <ClassMap: {'background': 0, 'decay': 1}>
	- Labels: [1, 1]
	- BBoxes: [<BBox (xmin:925.3009714419476, ymin:330.99206845622416, xmax:971.4576896067416, ymax:358.05555006295197)>, <BBox (xmin:641.5820458801498, ymin:696.3153812629132, xmax:674.8044826779026, ymax:740.6063020564308)>]


In [32]:
metric = COCOMetric(metric_type=COCOMetricType.bbox)
# metric.accumulate(preds_all)
metric.accumulate(preds_all_train)
metric_logs = metric.finalize()
metric_logs

{'AP (IoU=0.50:0.95) area=all': 0.4206618063233889,
 'AP (IoU=0.50) area=all': 0.8390982903035463,
 'AP (IoU=0.75) area=all': 0.36276139370847943,
 'AP (IoU=0.50:0.95) area=small': 0.31433458578593615,
 'AP (IoU=0.50:0.95) area=medium': 0.5088662939961903,
 'AP (IoU=0.50:0.95) area=large': 0.5349099922317117,
 'AR (IoU=0.50:0.95) area=all maxDets=1': 0.2506849315068493,
 'AR (IoU=0.50:0.95) area=all maxDets=10': 0.4970958904109589,
 'AR (IoU=0.50:0.95) area=all maxDets=100': 0.5206575342465752,
 'AR (IoU=0.50:0.95) area=small maxDets=100': 0.42742123687281214,
 'AR (IoU=0.50:0.95) area=medium maxDets=100': 0.6014739229024942,
 'AR (IoU=0.50:0.95) area=large maxDets=100': 0.6209302325581396}

In [13]:

from icevision.models.inference import process_bbox_predictions
# from eval_utils import inverse_transform, print_boxes
import eval_utils
import importlib
importlib.reload(eval_utils)

<module 'eval_utils' from '/mnt/home.stud/kuntluka/DetectionFramework/eval_utils.py'>

In [25]:
from pathlib import Path

from PIL import Image
data_path = Path("/home.stud/kuntluka/dataset/carries_dataset/images")
list(data_path.iterdir())
# data_path.

[Path('/home.stud/kuntluka/dataset/carries_dataset/images/355.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/880.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/734.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/384.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/423.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/851.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/1313.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/909.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/1465.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/1004.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/71.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/329.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/1541.png'),
 Path('/home.stud/kuntluka/dataset/carries_dataset/images/748.png'),
 Path('/home.stud/kuntluka/data

In [29]:

import json
# %reload_ext autoreload
# p7['detection']['bboxes']
# p7['scores']
# p7['bboxes']
# it = inverse_transform(1068, 847, 1024, p7.bboxes)
# from eval_utils import inerse_transform_preds, generate_detections_json
import eval_utils
import importlib
importlib.reload(eval_utils)

# print(it)
# p7.bboxes
# p7 = preds[7].detection
# p7
# print_boxes
# it
preds[7].as_dict()
print(preds[7].get_component_by_type(RecordComponent).as_dict())
# img = Image.open(data_path / preds[7].get_component_by_type(RecordComponent).as_dict()['record_id'])
# preds[7].ground_truth.detection.bboxes = eval_utils.inverse_transform(1068, 847, 1024, preds[7].ground_truth.detection.bboxes)
# eval_utils.print_boxes(img,preds[8].detection, preds[8].ground_truth.detection)
print(preds[7].ground_truth)
print(preds[7].detection)

# for i in range(10, 20):
#     img = Image.open(data_path / preds[i].common.record_id)
#     p, gt = eval_utils.inverse_transform_preds(preds[i].detection, preds[i].ground_truth.detection)
#     eval_utils.print_boxes(img, p, gt)

# d_sub = d['1363.png']
# d_sub['scores'] = d_sub['scores'].tolist()
# print(d_sub)
d = eval_utils.generate_detections_json(preds_all, preds2=preds_all_train)
with open('all_preds.json', 'w') as json_file:
    json.dump(d, json_file)

# d = eval_utils.generate_detections_json(preds_all)
# with open('training.json', 'w') as json_file:
#     json.dump(d, json_file)


{'record_id': '1363.png'}
BaseRecord

common: 
	- Record ID: 1363.png
	- Filepath: /home.stud/kuntluka/dataset/carries_dataset/images/1363.png
	- Img: None
	- Image size ImgSize(width=1024, height=1024)
detection: 
	- Class Map: <ClassMap: {'background': 0, 'decay': 1}>
	- Labels: [1, 1]
	- BBoxes: [<BBox (xmin:925.3009714419476, ymin:330.99206845622416, xmax:971.4576896067416, ymax:358.05555006295197)>, <BBox (xmin:641.5820458801498, ymin:696.3153812629132, xmax:674.8044826779026, ymax:740.6063020564308)>]


In [62]:
# preds[2].as_dict()
# preds[3].detection.as_dict()
preds[7].detection.reduce_on_components("as_dict")
# preds[7].ground_truth.detection.bboxes
# img = Image.open(preds[7].ground_truth.common.filepath)
# print(preds[7].ground_truth.common.filepath)

# tr = [*tfms.A.resize_and_pad(1024), tfms.A.Normalize()]
# proc = process_bbox_predictions(preds[7], img, tr)
# # proc.ad_dict()
# print(proc.components)
# # proc.detection.ground_truth
# # foo=proc.ground_truth.reduce_on_components("as_dict")
# # foo1=proc.detection.reduce_on_components("as_dict")
# # print(foo)
# print(foo1)
# foo2 = preds[7].detection.reduce_on_components("as_dict")
# print(foo2)
# preds[1].common.as_dict()
# preds[0]..as_dict()

[{'scores': array([    0.85559,     0.80779,     0.31861], dtype=float32)},
 {'labels': ['decay', 'decay', 'decay'], 'label_ids': [1, 1, 1]},
 {'bboxes': [<BBox (xmin:410, ymin:552, xmax:459, ymax:619)>,
   <BBox (xmin:713, ymin:110, xmax:778, ymax:182)>,
   <BBox (xmin:458, ymin:568, xmax:483, ymax:614)>]}]

In [133]:
print(img.size)

(1068, 847)


In [9]:
# val_ds[0].components
# p = val_ds[0].get_component_by_type(FilepathRecordComponent)
# id = val_ds[0].get_component_by_type(RecordIDRecordComponent)

# print(p.as_dict()['filepath'])
# print(id.as_dict())
val_ds = dm.valid_ds
train_ds = dm.train_ds
ids = []
for i in range(10):
    id = train_ds[i].get_component_by_type(RecordIDRecordComponent)
    ids.append(id.record_id)
print(ids)

['1013.png', '833.png', '1473.png', '886.png', '372.png', '1252.png', '731.png', '1098.png', '1571.png', '158.png']


val_dm = dm.val_ds